In [1]:
import tensorflow as tf
import importlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from datetime import datetime
import utils_triplet2
import model2
import results_generator
importlib.reload(utils_triplet2)
importlib.reload(model2)
importlib.reload(results_generator)
from utils_triplet2 import *
from model2 import *
from results_generator import *
tf.test.is_built_with_cuda()

True

In [2]:
# This code is only needed if the dataset is stored in google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Check if gpu exists
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [6]:
# This code is only needed if the dataset is stored in google drive and it is 7z compressed
!p7zip -d /content/gdrive/MyDrive/Thesis/Sketch-Icon-Dataset.7z


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/gdrive/MyDrive/Thesis/                                         1 file, 263683120 bytes (252 MiB)

Extracting archive: /content/gdrive/MyDrive/Thesis/Sketch-Icon-Dataset.7z
--
Path = /content/gdrive/MyDrive/Thesis/Sketch-Icon-Dataset.7z
Type = 7z
Physical Size = 263683120
Headers Size = 344455
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      1% 952 - Sketch-Icon-Dataset/icon/mathematics/wave-square.jpg                                                                 1% 1111 - Sketch-Icon-Dataset/icon/science-fiction/old-repub

In [7]:
icon_name_category, sketch_name_category = get_icons_and_sketches()

In [8]:
# I dont split the data into train and test because I want all of the icons to be trained, since the icons dataset will always be the same
icon_categories =  np.unique(icon_name_category[:, 1], return_index=False)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(icon_categories)

# binary encode the classes of icons
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
icon_categories_dic = {}
for i in range(len(icon_categories)):
    icon_categories_dic[icon_categories[i]] = onehot_encoded[i]

# len_data = len(icon_name_category)
# p_train=0.9
# p_test=0.1
# num_train = int(np.ceil(len_data*p_train))
# num_test = int(np.floor(len_data*p_test))

# icon_name_category = shuffle(icon_name_category)
# icons_Train = icon_name_category[:num_train]
# icons_Test= icon_name_category[-num_test:]

# print(f'We have {len(icons_Train)} samples in the training set.')
# print(f'We have {len(icons_Test)} samples in the test set.')

In [9]:
# Creation of folders for storing stats, weights and the hyperparameters of the model
model_name = "GoogleNet"
algorithm_name = "Icon Classification"
create_directory(model_name + "/")
now = datetime.now()
date_time_folder = now.strftime("%d-%m-%Y %H-%M-%S")
train_weights_folder = "Train Weights"
create_directory(model_name + "/" + algorithm_name)
current_run_path = model_name + "/" + algorithm_name + "/" + date_time_folder + "/"
create_directory(current_run_path)
train_weights_path = model_name + "/" + algorithm_name + "/" + date_time_folder + "/" + train_weights_folder
create_directory(train_weights_path)
#=====================================================================================

# Hyperparameter initializations
BATCH_SIZE = 128
num_epochs = 100
learing_rate = 0.001

# Store the hyperparameters of the model into a file to remember what I used in a specific run
write_classification_hyperparameters_in_file(current_run_path, learing_rate, BATCH_SIZE)
optimizer = tf.keras.optimizers.Adam(learing_rate)
tf.keras.backend.set_floatx('float32')
iconClassificationModel = googlenet(len(icon_categories))
loss = compute_cross_entropy

# Function which is responsible to calculate the loss and update the grads
def train_step(icons, targets):
    with tf.GradientTape() as tape:
        outputs = iconClassificationModel(icons, training = True)
        main_output = outputs[0]
        aux1_output = outputs[1]
        aux2_output = outputs[2]
        tape.watch(main_output)
        tape.watch(aux1_output)
        tape.watch(aux2_output)
        current_loss_main = loss(main_output, targets)
        current_loss_aux1 = loss(aux1_output, targets)
        current_loss_aux2 = loss(aux2_output, targets)
        current_loss = current_loss_main + 0.3 * current_loss_aux1 + 0.3 * current_loss_aux2
    grads = tape.gradient(current_loss, iconClassificationModel.trainable_variables)
    optimizer.apply_gradients(zip(grads, iconClassificationModel.trainable_variables))
    return current_loss

Successfully created the directory  GoogleNet/
Successfully created the directory  GoogleNet/Icon Classification
Successfully created the directory  GoogleNet/Icon Classification/01-04-2021 15-57-36/
Successfully created the directory  GoogleNet/Icon Classification/01-04-2021 15-57-36/Train Weights


In [10]:
top_acc = 0
for epoch in range(num_epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    #  Training loop throught the dataset
    for i in range(0, len(icon_name_category), BATCH_SIZE):
        icon_name_category_batch = icon_name_category[i:i+BATCH_SIZE]
        icons, targets = get_batch_icons_targets_low_ram(icon_name_category_batch, icon_categories_dic)
        loss_value = train_step(icons, targets)
        epoch_loss_avg.update_state(loss_value)
    print("Epoch {:d}: Loss: {:.3f}".format(epoch,epoch_loss_avg.result()))

    # Test loop through the same datatet
    acc = 0
    for i in range(0, len(icon_name_category), BATCH_SIZE):
        icon_name_category_batch = icon_name_category[i:i+BATCH_SIZE]
        icons, targets = get_batch_icons_targets_low_ram(icon_name_category_batch, icon_categories_dic)
        outputs_main = iconClassificationModel(icons, training = False)[0] # take only the main output
        correct_prediction = tf.equal(tf.argmax(outputs_main,1),tf.argmax(targets,1))
        for prediction in correct_prediction:
            if prediction:
                acc = acc + 1

    # Storing of model weights if the accuracy is better than the current max accuracy
    if top_acc<acc:
        top_acc = acc
        save_weights(iconClassificationModel, train_weights_path + "/iconClassification")

    # Storing of epoch, loss and accuracy into a file
    write_classification_stats_in_file(current_run_path, epoch, epoch_loss_avg.result(), acc/len(icon_name_category))
    print("Test accuracy is: " + str(acc/len(icon_name_category)))


Epoch 0: Loss: 7.211
Test accuracy is: 0.06112054329371817


KeyboardInterrupt: ignored

## Sketch Classification

In [11]:
sketch_categories =  np.unique(sketch_name_category[:, 1], return_index=False)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(sketch_categories)

# binary encode the classes of sketches
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
sketch_categories_dic = {}
for i in range(len(sketch_categories)):
    sketch_categories_dic[sketch_categories[i]] = onehot_encoded[i]

# Split the dataset into train and test with randomness
# or load the train and test dataset which is created as numpy arrays.
# The purpose of the predefined datasets is to have the same datasets for training
# the classification of sketches and the sketch-icon retrieval, so the test set will be the same,
# and unknown for both trainings
random_generation = False
if random_generation:
    len_data = len(sketch_name_category)
    p_train=0.9
    p_test=0.1
    num_train = int(np.ceil(len_data*p_train))
    num_test = int(np.floor(len_data*p_test))

    sketch_name_category = shuffle(sketch_name_category)
    sketches_Train = sketch_name_category[:num_train]
    sketches_Test= sketch_name_category[-num_test:]
else:
    sketches_Train = load_train_set()
    sketches_Train = shuffle(sketches_Train)
    sketches_Test = load_test_set()

print(f'We have {len(sketches_Train)} samples in the training set.')
print(f'We have {len(sketches_Test)} samples in the test set.')

We have 28626 samples in the training set.
We have 3180 samples in the test set.


In [12]:
# Creation of folders for storing stats, weights and the hyperparameters of the model
model_name = "GoogleNet"
algorithm_name = "Sketch Classification"
create_directory(model_name + "/")
now = datetime.now()
date_time_folder = now.strftime("%d-%m-%Y %H-%M-%S")
train_weights_folder = "Train Weights"
create_directory(model_name + "/" + algorithm_name)
current_run_path = model_name + "/" + algorithm_name + "/" + date_time_folder + "/"
create_directory(current_run_path)
train_weights_path = model_name + "/" + algorithm_name + "/" + date_time_folder + "/" + train_weights_folder
create_directory(train_weights_path)
#=====================================================================================

# Hyperparameter initializations
BATCH_SIZE = 128
num_epochs = 300
learing_rate = 0.001

# Store the hyperparameters of the model into a file to remember what I used in a specific run
write_classification_hyperparameters_in_file(current_run_path, learing_rate, BATCH_SIZE)

# Initializations of optimizer, loss and network and set the backend (model variables) to float32
optimizer = tf.keras.optimizers.Adam(learing_rate)
tf.keras.backend.set_floatx('float32')
sketchClassificationModel = googlenet(len(sketch_categories))
loss = compute_cross_entropy

# Function which is responsible to calculate the loss and update the grads
def train_step(sketches, targets):
    with tf.GradientTape() as tape:
        outputs = sketchClassificationModel(sketches, training = True)
        main_output = outputs[0]
        aux1_output = outputs[1]
        aux2_output = outputs[2]
        tape.watch(main_output)
        tape.watch(aux1_output)
        tape.watch(aux2_output)
        current_loss_main = loss(main_output, targets)
        current_loss_aux1 = loss(aux1_output, targets)
        current_loss_aux2 = loss(aux2_output, targets)
        current_loss = current_loss_main + 0.3 * current_loss_aux1 + 0.3 * current_loss_aux2
    grads = tape.gradient(current_loss, sketchClassificationModel.trainable_variables)
    optimizer.apply_gradients(zip(grads, sketchClassificationModel.trainable_variables))
    return current_loss


Creation of the directory  failedGoogleNet/
Successfully created the directory  GoogleNet/Sketch Classification
Successfully created the directory  GoogleNet/Sketch Classification/01-04-2021 15-58-46/
Successfully created the directory  GoogleNet/Sketch Classification/01-04-2021 15-58-46/Train Weights


In [13]:

top_acc = 0
for epoch in range(num_epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    
    # Training loop throught the training dataset
    for i in range(0, len(sketches_Train), BATCH_SIZE):
        sketch_name_category_batch = sketches_Train[i:i+BATCH_SIZE]
        sketches, targets = get_batch_sketches_targets_low_ram(sketch_name_category_batch, sketch_categories_dic)
        loss_value = train_step(sketches, targets)
        epoch_loss_avg.update_state(loss_value)
    print("Epoch {:d}: Loss: {:.3f}".format(epoch,epoch_loss_avg.result()))

    # Test loop through the test datatet
    acc = 0
    for i in range(0, len(sketches_Test), BATCH_SIZE):
        sketch_name_category_batch = sketches_Test[i:i+BATCH_SIZE]
        sketches, targets = get_batch_sketches_targets_low_ram(sketch_name_category_batch, sketch_categories_dic)
        outputs_main = sketchClassificationModel(sketches, training = False)[0]
        correct_prediction = tf.equal(tf.argmax(outputs_main,1),tf.argmax(targets,1))
        for prediction in correct_prediction:
            if prediction:
                acc = acc + 1

    # Storing of model weights if the accuracy is better than the current max accuracy
    if top_acc<acc:
        top_acc = acc
        save_weights(sketchClassificationModel, train_weights_path + "/sketchClassification")
    print("Test accuracy is: " + str(acc/len(sketches_Test)))

    # Storing of epoch, loss and accuracy into a file
    write_classification_stats_in_file(current_run_path, epoch, epoch_loss_avg.result(), acc/len(sketches_Test))

    # shuffle the training set every epoch
    sketches_Train = shuffle(sketches_Train)
print("The best accuracy is: " + str(top_acc/len(sketches_Test)))

Epoch 0: Loss: 6.280
Test accuracy is: 0.03647798742138365


KeyboardInterrupt: ignored